In [3]:
from __future__ import print_function
import mxnet as mx
import numpy as np
import glob
import os
from mxnet.contrib.onnx import import_model
from cityscapes_loader import CityLoader

In [5]:
# Determine and set context
if len(mx.test_utils.list_gpus())==0:
    ctx = mx.cpu()
else:
    ctx = mx.gpu(0)

# Path to validation data
data_dir = '/Users/arvindkumarsingh/Desktop/Oneapi/data/leftImg8bit/val'
# Path to validation labels
label_dir = '/Users/arvindkumarsingh/Desktop/Oneapi/data//gtFine/val'
# Set batch size
batch_size = 16

In [6]:
# mx.test_utils.download('https://s3.amazonaws.com/onnx-model-zoo/duc/ResNet101_DUC_HDC.onnx')
# Path to ONNX model
model_path = 'ResNet101_DUC_HDC.onnx'

In [7]:
index = 0
val_lst = []
# images
all_images = glob.glob(os.path.join(data_dir, '*/*.png'))
all_images.sort()
for p in all_images:
    l = p.replace(data_dir, label_dir).replace('leftImg8bit', 'gtFine_labelIds')
    if os.path.isfile(l):
        index += 1
        for i in range(1, 8):
            val_lst.append([str(index), p, l, "512", str(256 * i)])

val_out = open('val.lst', "w")
for line in val_lst:
    print('\t'.join(line),file=val_out)

In [8]:
def check_label_shapes(labels, preds, shape=0):
    if shape == 0:
        label_shape, pred_shape = len(labels), len(preds)
    else:
        label_shape, pred_shape = labels.shape, preds.shape

    if label_shape != pred_shape:
        raise ValueError("Shape of labels {} does not match shape of "
                         "predictions {}".format(label_shape, pred_shape))

class IoUMetric(mx.metric.EvalMetric):
    def __init__(self, ignore_label, label_num, name='IoU'):
        self._ignore_label = ignore_label
        self._label_num = label_num
        super(IoUMetric, self).__init__(name=name)

    def reset(self):
        self._tp = [0.0] * self._label_num
        self._denom = [0.0] * self._label_num

    def update(self, labels, preds):
        check_label_shapes(labels, preds)
        for i in range(len(labels)):
            pred_label = mx.ndarray.argmax_channel(preds[i]).asnumpy().astype('int32')
            label = labels[i].asnumpy().astype('int32')

            check_label_shapes(label, pred_label)

            iou = 0
            eps = 1e-6
            for j in range(self._label_num):
                pred_cur = (pred_label.flat == j)
                gt_cur = (label.flat == j)
                tp = np.logical_and(pred_cur, gt_cur).sum()
                denom = np.logical_or(pred_cur, gt_cur).sum() - np.logical_and(pred_cur, label.flat == self._ignore_label).sum()
                assert tp <= denom
                self._tp[j] += tp
                self._denom[j] += denom
                iou += self._tp[j] / (self._denom[j] + eps)
            iou /= self._label_num
            self.sum_metric = iou
            self.num_inst = 1

            
# Create evaluation metric
met = IoUMetric(ignore_label=255, label_num=19, name="IoU")
metric = mx.metric.create(met)

In [9]:
loader = CityLoader
val_args = {
    'data_path'             : data_dir,
    'label_path'            : label_dir,
    'rgb_mean'              : (122.675, 116.669, 104.008),
    'batch_size'            : batch_size,
    'scale_factors'         : [1],
    'data_name'             : 'data',
    'label_name'            : 'seg_loss_label',
    'data_shape'            : [tuple(list([batch_size, 3, 800, 800]))],
    'label_shape'           : [tuple([batch_size, (160000)])],
    'use_random_crop'       : False,
    'use_mirror'            : False,
    'ds_rate'               : 8,
    'convert_label'         : True,
    'multi_thread'          : False,
    'cell_width'            : 2,
    'random_bound'          : [120,120],
}
val_dataloader = loader('val.lst', val_args)

In [10]:
# import ONNX model into MXNet symbols and params
sym,arg,aux = import_model(model_path)
# define network module
mod = mx.mod.Module(symbol=sym, data_names=['data'], context=ctx, label_names=None)
# bind parameters to the network
mod.bind(for_training=False, data_shapes=[('data', (batch_size, 3, 800, 800))], label_shapes=mod._label_shapes)
mod.set_params(arg_params=arg, aux_params=aux,allow_missing=True, allow_extra=True)

Calling mxnet.contrib.onnx.import_model...
Please be advised that importing ONNX models into MXNet is going to be deprecated in the upcoming MXNet v1.10 release. The following apis will be deleted: mxnet.contrib.onnx.import_model/get_model_metadata/import_to_gluon.


[14:51:26] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.


In [ ]:
# reset data loader
val_dataloader.reset()
# reset evaluation metric
metric.reset()
# loop over batches
for nbatch, eval_batch in enumerate(val_dataloader):
    # perform forward pass
    mod.forward(eval_batch, is_train=False)
    # get outputs
    outputs=mod.get_outputs()
    # update evaluation metric
    metric.update(eval_batch.label,outputs)
    # print progress
    if nbatch%10==0:
        print('{} / {} batches done'.format(nbatch,int(3500/batch_size)))

0 / 218 batches done


In [ ]:
print("mean Intersection Over Union (mIOU): {}".format(metric.get()[1]))